In [1]:
# ✅ Step 0: Imports
import os
os.environ["OMP_NUM_THREADS"] = "2"  # (Optional) Limit CPU thread usage

import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import resample

# ✅ Step 1: Load data and flatten labels
data = np.load("C:/Users/ishsi/Downloads/pneumoniamnist.npz")
x_train, y_train = data["train_images"], data["train_labels"]
x_val, y_val = data["val_images"], data["val_labels"]
x_test, y_test = data["test_images"], data["test_labels"]

y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()

print(f"Train: {x_train.shape}, Val: {x_val.shape}, Test: {x_test.shape}")
print("Class 0:", np.sum(y_train == 0), "Class 1:", np.sum(y_train == 1))

# ✅ Step 2: Oversample class 0
x_train_0 = x_train[y_train == 0]
x_train_1 = x_train[y_train == 1]
y_train_0 = y_train[y_train == 0]
y_train_1 = y_train[y_train == 1]

x_train_0_up, y_train_0_up = resample(
    x_train_0, y_train_0,
    replace=True,
    n_samples=len(y_train_1),
    random_state=42
)

x_train_bal = np.concatenate([x_train_0_up, x_train_1])
y_train_bal = np.concatenate([y_train_0_up, y_train_1])

shuffle_idx = np.random.permutation(len(y_train_bal))
x_train_bal = x_train_bal[shuffle_idx]
y_train_bal = y_train_bal[shuffle_idx]

print("Balanced counts:", np.bincount(y_train_bal.astype(int)))

# ✅ Step 3: Preprocessing function
def preprocess(x, y):
    x = tf.image.resize(x[..., tf.newaxis], (224, 224))
    x = tf.image.grayscale_to_rgb(x)
    x = tf.cast(x, tf.float32) / 255.0
    return x, y

# ✅ Step 4: Create datasets with batch size 8
batch_size = 8

train_ds = tf.data.Dataset.from_tensor_slices((x_train_bal, y_train_bal))
train_ds = train_ds.shuffle(1000).map(preprocess).map(lambda x, y: (tf.image.random_flip_left_right(x), y)).batch(batch_size)

val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).map(preprocess).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocess).batch(batch_size)

# ✅ Step 5: Custom Focal Loss with logits
def binary_focal_loss(gamma=2.0, alpha=0.25, from_logits=False):
    def loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        if from_logits:
            y_pred = tf.nn.sigmoid(y_pred)
        eps = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, eps, 1. - eps)
        p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        alpha_factor = tf.where(tf.equal(y_true, 1), alpha, 1 - alpha)
        modulating_factor = tf.pow((1 - p_t), gamma)
        return tf.reduce_mean(-alpha_factor * modulating_factor * tf.math.log(p_t))
    return loss_fn

# ✅ Step 6: Build model using MobileNetV2
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze for initial training

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)  # No sigmoid (logits output)
])

model.compile(
    optimizer='adam',
    loss=binary_focal_loss(gamma=2.0, alpha=0.25, from_logits=True),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# ✅ Step 7: Train
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

# ✅ Step 8: Debug prediction distribution
val_probs = tf.nn.sigmoid(model.predict(val_ds)).numpy()
val_preds = (val_probs > 0.5).astype(int)
val_true = np.concatenate([y for _, y in val_ds], axis=0)

print("🔍 Validation Predictions:")
print("Predicted:", np.bincount(val_preds.flatten()))
print("Actual:", np.bincount(val_true.astype(int)))

# ✅ Step 9: Fine-tune top layers
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),
    loss=binary_focal_loss(gamma=2.0, alpha=0.25, from_logits=True),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

# ✅ Step 10: Evaluation
print("\n📊 Train Set Evaluation:")
model.evaluate(train_ds)

print("\n📊 Test Set Evaluation:")
model.evaluate(test_ds)

# ✅ Step 11: Test metrics
test_probs = tf.nn.sigmoid(model.predict(test_ds)).numpy()
test_preds = (test_probs > 0.5).astype(int)
test_true = np.concatenate([y for _, y in test_ds], axis=0)

print("\n📈 Classification Report (Test):")
print(classification_report(test_true, test_preds))
print("📉 Confusion Matrix:")
print(confusion_matrix(test_true, test_preds))
print("🧠 ROC AUC Score:", roc_auc_score(test_true, test_probs))


Train: (3882, 28, 28), Val: (524, 28, 28), Test: (624, 28, 28)
Class 0: 388 Class 1: 3494
Balanced counts: [3494 3494]
9406464/9406464 [==============================] - 20s 2us/step
Epoch 1/10
874/874 [==============================] - 104s 111ms/step - loss: 0.0294 - accuracy: 0.8520 - precision: 0.9839 - recall: 0.7158 - val_loss: 0.0247 - val_accuracy: 0.8225 - val_precision: 0.9933 - val_recall: 0.7661 - lr: 0.0010
Epoch 2/10
874/874 [==============================] - 100s 114ms/step - loss: 0.0207 - accuracy: 0.8706 - precision: 0.9905 - recall: 0.7484 - val_loss: 0.0193 - val_accuracy: 0.8569 - val_precision: 0.9906 - val_recall: 0.8149 - lr: 0.0010
Epoch 3/10
874/874 [==============================] - 100s 115ms/step - loss: 0.0188 - accuracy: 0.8891 - precision: 0.9917 - recall: 0.7848 - val_loss: 0.0194 - val_accuracy: 0.8206 - val_precision: 1.0000 - val_recall: 0.7584 - lr: 0.0010
Epoch 4/10
874/874 [==============================] - 95s 109ms/step - loss: 0.0171 - accuracy